In [ ]:
#hide
from public_data_food_analysis_3 import *

# Public data food analysis 

> 

## Install

`pip install public_data_food_analysis_3`

## For data analysis on the Columbia study

In [ ]:
import public_data_food_analysis_3.columbia as pdfac
import pandas as pd

In [ ]:
df = pdfac.make_table(pdfac.read_logging_data('data/col_test_data')\
                      , pd.read_excel('data/col_test_data/toy_data_17May2021.xlsx'))

Participant yrt1999 didn't log any food items in the following day(s):
2021-05-18
Participant yrt2000 didn't log any food items in the following day(s):
2021-05-12
2021-05-13
2021-05-14


In [ ]:
df

,mCC_ID,Participant_Study_ID,Study Phase,Intervention group (TRE or HABIT),Start_Day,End_day,Eating_Window_Start,Eating_Window_End,phase_duration,caloric_entries,...,logging_day_counts,%_logging_day_counts,good_logging_days,%_good_logging_days,good_window_days,%_good_window_days,outside_window_days,%_outside_window_days,adherent_days,%_adherent_days
0,yrt1999,2,S-REM,TRE,2021-05-12,2021-05-14,00:00:00,23:59:00,3 days,7,...,3,1.00,2.0,0.666667,3,1.00,0,0.0,2,0.666667
1,yrt1999,2,T3-INT,TRE,2021-05-15,2021-05-18,08:00:00,18:00:00,4 days,8,...,3,0.75,2.0,0.500000,1,0.25,2,0.5,1,0.250000
2,yrt2000,3,T3-INT,TRE,2021-05-12,2021-05-14,08:00:00,16:00:00,3 days,0,...,0,0.00,0.0,0.000000,0,0.00,0,0.0,0,0.000000


In [ ]:
df.iloc[0]

mCC_ID                                           yrt1999
Participant_Study_ID                                   2
Study Phase                                        S-REM
Intervention group (TRE or HABIT)                    TRE
Start_Day                            2021-05-12 00:00:00
End_day                              2021-05-14 00:00:00
Eating_Window_Start                             00:00:00
Eating_Window_End                               23:59:00
phase_duration                           3 days 00:00:00
caloric_entries                                        7
mean_daily_eating_window                           13.75
std_daily_eating_window                        11.986972
earliest_entry                                       4.5
2.5%                                              4.5375
97.5%                                            27.5625
duration mid 95%                                  23.025
logging_day_counts                                     3
%_logging_day_counts           

In [ ]:
df.iloc[1]

mCC_ID                                           yrt1999
Participant_Study_ID                                   2
Study Phase                                       T3-INT
Intervention group (TRE or HABIT)                    TRE
Start_Day                            2021-05-15 00:00:00
End_day                              2021-05-18 00:00:00
Eating_Window_Start                             08:00:00
Eating_Window_End                               18:00:00
phase_duration                           4 days 00:00:00
caloric_entries                                        8
mean_daily_eating_window                        8.666667
std_daily_eating_window                         8.504901
earliest_entry                                       7.5
2.5%                                                 7.7
97.5%                                               23.9
duration mid 95%                                    16.2
logging_day_counts                                     3
%_logging_day_counts           